In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts  import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature = 0.1,

)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer questions usting only the following context. If you don't know the answer just say you don't know, don't make it up:\n{context}"),
    ("human", "{question}")
])

chain = {"context":retriever, "question": RunnablePassthrough()} | prompt |  llm

chain.invoke("Describe Seolhee")

AIMessage(content="Seolhee Kim is a dedicated and detail-oriented IT Application Developer. She has a strong educational background, having graduated from Niagara College's Computer Programming and Analysis program. Seolhee has a solid command of both front-end and back-end programming, as well as computer science concepts. She has received recognition for her outstanding academic performance and has successfully completed various projects, including an inventory application chosen by CAA and a web-based dashboard for a client.\n\nSeolhee is an active collaborator and enjoys sharing her ideas and incorporating new solutions to challenges. She has developed problem-solving skills and actively contributes to group projects using GitHub. She is also involved in the tech community as a Microsoft Student Ambassador and has volunteered to teach Korean utilizing AI software at Niagara Korean School. Seolhee's time management and organizational skills allow her to effectively manage multiple p